<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: [Available Here](https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
url = "https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv"
telecom = pd.read_csv(url)
telecom.shape

(7043, 21)

In [8]:
telecom.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


### Feature Engineering and Encoding
**Necessary Changes**
- The `customerID` column has too many unique values

In [4]:
from sklearn.model_selection import train_test_split

# Split the data into train and test
train, test = train_test_split(telecom, test_size=0.20, random_state=42)

print(train.shape, test.shape)

(5634, 21) (1409, 21)


In [33]:
train.describe(exclude=np.number).T.sort_values(by='unique', ascending=False)

,count,unique,top,freq
customerID,5634,5634,7951-QKZPL,1
TotalCharges,5634,5300,,10
PaymentMethod,5634,4,Electronic check,1887
OnlineBackup,5634,3,No,2448
Contract,5634,3,Month-to-month,3083
StreamingMovies,5634,3,No,2223
StreamingTV,5634,3,No,2239
TechSupport,5634,3,No,2791
DeviceProtection,5634,3,No,2489
OnlineSecurity,5634,3,No,2810


In [7]:
def wrangle(data):
    """Clean data and engineer features"""
    df = data.copy()
    
    # Change TotalCharges to float
    df["TotalCharges"].loc[df["TotalCharges"] == ' '] = np.nan
    df["TotalCharges"] = pd.to_numeric(df["TotalCharges"])
    df["TotalCharges"] = df["TotalCharges"].fillna(df["TotalCharges"].mean())
    
    # Drop customerID
    df = df.drop(columns=["customerID"])
    
    # Convert target to zeros and ones
    df["Churn"] = df["Churn"].map({"Yes": 1, "No": 0})
    
    return df

In [8]:
train = wrangle(train)
test = wrangle(test)

In [122]:
# Arrange data into X and y form
target = "Churn"
features = train.drop(columns=[target]).columns

X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

X_train.shape, X_test.shape

((5634, 19), (1409, 19))

In [123]:
from category_encoders import OneHotEncoder
from sklearn.preprocessing import StandardScaler

encoder = OneHotEncoder()
scaler = StandardScaler()

X_train = encoder.fit_transform(X_train)
X_test = encoder.transform(X_test)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [124]:
import tensorflow as tf
from tensorflow import keras

def build_model(n_hidden=2, n_neurons=32, learning_rate=3e-3, input_shape=[45]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1)) # Try without
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=['acc'])
    return model

In [125]:
# Classifier wrapper
keras_clf = keras.wrappers.scikit_learn.KerasClassifier(build_fn=build_model, verbose=False)

In [128]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    "batch_size": [30, 40, 50, 60],
    "learning_rate": [0.008, 0.01, 0.015],
    "epochs": [10, 20, 30]
}

rand_search = RandomizedSearchCV(keras_clf, params, n_iter=10, cv=3)
rand_search.fit(X_train, y_train, epochs=100, validation_data=(X_test, y_test),
                callbacks=[keras.callbacks.EarlyStopping(patience=10)])

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=<tensorflow.python.keras.wrappers.scikit_learn.KerasClassifier object at 0x7fa0b98974e0>,
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'batch_size': [30, 40, 50, 60],
                                        'epochs': [10, 20, 30],
                                        'learning_rate': [0.008, 0.01, 0.015]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [133]:
rand_search.best_params_

{'learning_rate': 0.008, 'epochs': 10, 'batch_size': 60}

## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?